The following code defines hyperparameter tuning for LSTM model designed for uni-modal dataset. 

In [ ]:
import pandas as pd
import os
import numpy as np
import json

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [ ]:
scaler = StandardScaler()

## Data preparation

### X data - audio and visual features

In [ ]:
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - YMRS score

In [ ]:
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

## Hyperparameter Tuner setup

In [ ]:
def model_builder_visual(hp):
    
    model = keras.Sequential()
    
    # Hyperparameters for tuning
    hp_dense_units_1 = hp.Int('units_1', min_value=1, max_value=465, step=1)
    hp_dense_units_2 = hp.Int('units_2', min_value=1, max_value=465, step=1)
    hp_droput = hp.Choice('dropout', values=[1e-2, 1e-3, 1e-4])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate for the optimizer
    
    model.add(layers.LSTM(
                        units = hp_dense_units_1, 
                        input_shape=(None,465), 
                        return_sequences=True 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.LSTM(
                        units = hp_dense_units_2, 
                        input_shape=(None,hp_dense_units_1), 
                        return_sequences=False 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', 
                  metrics=[keras.metrics.MeanAbsoluteError()] ,
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  ) 
    
    return model

In [ ]:
def model_builder_audio(hp):
    model = keras.Sequential()
    
    hp_dense_units_1 = hp.Int('units_1', min_value=1, max_value=23, step=1)
    hp_dense_units_2 = hp.Int('units_2', min_value=1, max_value=23, step=1)
    hp_droput = hp.Choice('dropout', values=[1e-2, 1e-3, 1e-4])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate for the optimizer
    
    model.add(layers.LSTM(
                        units = hp_dense_units_1, 
                        input_shape=(None,23), 
                        return_sequences=True 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.LSTM(
                        units = hp_dense_units_2, 
                        input_shape=(None,hp_dense_units_1), 
                        return_sequences=False 
                    ))
    model.add(layers.Dropout(hp_droput))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(loss='mse', 
                  metrics=[keras.metrics.MeanAbsoluteError()] ,
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  ) 
    
    return model

In [ ]:
#Initilize tuners - for video and audio modality seperately

tuner_visual = kt.Hyperband(
    model_builder_visual,
    objective = 'loss',
    max_epochs = 50,
    factor = 3,
    project_name = 'Hp_tuner_visual'
)

tuner_audio = kt.Hyperband(
    model_builder_audio,
    objective = 'loss',
    max_epochs = 50, 
    factor = 3,
    project_name = 'Hp_tuner_audio'
)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
# Train audio data tuner
index = 0
for file in x_train_audio:
    file = reshape_X(file)
    tuner_audio.search(
        file, 
        y_train[index], 
        epochs=10,  
        batch_size = len(file), 
        callbacks=[stop_early]) 

In [ ]:
#Train visual data tuner
index = 0
for file in x_train_visual:
    file = reshape_X(file)
    tuner_visual.search(
        file, 
        y_train[index], 
        epochs=10, 
        batch_size = len(file),
        callbacks=[stop_early])

### Finding the final hyperparameter values

The approach of finding the final hyperparameters for the model is based on extracting the average value from all tuner search iterations for each modality seperately. 

In [ ]:
# Combining learned hyperparameters from every iteration in one dataset

def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [ ]:
#Get hyperparameter values for both modalities
hpdf_visual = get_hp_df('../LSTM/Hp_tuner_visual')
hpdf_audio = get_hp_df('../LSTM/Hp_tuner_audio')

In [ ]:
print(""" Hyperparameters for the LSTM model for each data modality.
--------FOR VISUAL DATA:""")
print(hpdf_visual.mean())

print("--------FOR AUDIO DATA:")
print(hpdf_audio.mean())